In [2]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

1.12.0


In [3]:
X=np.random.random([10000,2])
W=np.array([3,4]).reshape([2,1])
Y=np.add(np.matmul(X,W),np.random.uniform(-0.5,0.5))

In [4]:
Y

array([[4.87153226],
       [2.14455232],
       [1.35547239],
       ...,
       [5.67853417],
       [1.95774712],
       [5.13755597]])

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
X_train.shape

(8000, 2)

In [7]:
X=tf.placeholder(tf.float32,shape=(None,2),name='modelInput_X')
Y=tf.placeholder(tf.float32,shape=(None,1),name='modelInput_Y')
W=tf.Variable(tf.zeros([2,1],dtype=tf.float32),name='modelWeights')
b=tf.Variable(np.random.randn,dtype=tf.float32,name='modelBias')
init=tf.global_variables_initializer()
pred=tf.add(tf.matmul(X,W),b,name='modelPred')
loss=tf.reduce_mean(tf.square(Y-pred),name='modelOutput')

In [8]:
optimize=tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [9]:
epochs = 10000
SAVED_MODEL_PATH='check_point/'
MODEL_NAME='Regession_Linear'

In [10]:
sess=tf.Session()
saver = tf.train.Saver()
sess.run(init)
for epoch in range(epochs+1) :
    sess.run(optimize,feed_dict={X:X_train,Y:Y_train})
    if epoch%1000==0 :
        var_loss=sess.run(loss,feed_dict={X:X_test,Y:Y_test})
        print("loss:",var_loss)
        out = saver.save(sess,SAVED_MODEL_PATH + MODEL_NAME+'.ckpt', global_step=epoch,write_meta_graph=False)
    

loss: 20.508692
loss: 0.07742852
loss: 0.008743742
loss: 0.0009942508
loss: 0.000113317015
loss: 1.2925942e-05
loss: 1.476947e-06
loss: 1.7026012e-07
loss: 1.981118e-08
loss: 2.124163e-09
loss: 9.474965e-10


In [12]:
sess.run(W)

array([[2.9999266],
       [3.9999247]], dtype=float32)

In [13]:
# Saving 
tf.train.write_graph(sess.graph_def, SAVED_MODEL_PATH , MODEL_NAME + '.pbtxt')
#tf.train.write_graph(sess.graph_def, SAVED_MODEL_PATH , MODEL_NAME + '.pb',as_text=True)

'check_point/Regession_Linear.pbtxt'

In [14]:
from tensorflow.python.tools import freeze_graph

# Freeze the graph

# graph definition saved above
input_graph = SAVED_MODEL_PATH+MODEL_NAME+'.pbtxt'
# any other saver to use other than default
input_saver = ""
# earlier definition file format text or binary
input_binary = False
# checkpoint file to merge with graph definition
input_checkpoint = SAVED_MODEL_PATH+MODEL_NAME+'.ckpt-'+str(10000)
# input nodes inn our model
input_node_names = 'modelInput_X'
# output nodes inn our model
output_node_names = 'modelPred'
restore_op_name = 'save/restore_all'
filename_tensor_name = 'save/Const:0'
# output path
output_graph = SAVED_MODEL_PATH+'frozen_'+MODEL_NAME+'.pb'
# default True
clear_devices = True
initializer_nodes = ""
variable_names_blacklist = ""

freeze_graph.freeze_graph(
    input_graph,
    input_saver,
    input_binary,
    input_checkpoint,
    output_node_names,
    restore_op_name,
    filename_tensor_name,
    output_graph,
    clear_devices,
    initializer_nodes,
    variable_names_blacklist
)

Instructions for updating:
Use tf.gfile.GFile.
INFO:tensorflow:Restoring parameters from check_point/Regession_Linear.ckpt-10000
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.


In [15]:
from tensorflow.python.tools import optimize_for_inference_lib
inputGraph = tf.GraphDef()
with tf.gfile.Open(SAVED_MODEL_PATH+'frozen_'+MODEL_NAME+'.pb', "rb") as f:
    data2read = f.read()
    inputGraph.ParseFromString(data2read)
    outputGraph = optimize_for_inference_lib.optimize_for_inference(
              inputGraph,
              ["modelInput_X"], # an array of the input node(s)
              ["modelPred"], # an array of output nodes
              tf.int32.as_datatype_enum)

# Save the optimized graph'test.pb'
f = tf.gfile.FastGFile(SAVED_MODEL_PATH+'optimize_'+MODEL_NAME+'.pb', "w")
f.write(outputGraph.SerializeToString()) 